# Test the algorithm for calculating distances between XRBs and clusters

Also calculate the velocity of ejection and make kde plots

In [2]:
import numpy as np
import pandas as pd

from XRBID.DataFrameMod import Find
from XRBID.Sources import Crossref, GetCoords

from astropy.io import fits

import os, sys
sys.path.insert(0, '/Users/undergradstudent/Research/XRB-Analysis/Notebooks')
cd = os.chdir

from helpers.analysis import remove_unnamed, calculate_distances #, euclidean_distance
from helpers.regions import WriteReg

hstdir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/HST/"
chandra_hst_dir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-HST/"
chandra_jwst_dir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/Chandra-JWST/"
jwstdir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/JWST/"
M66_dir = "/Users/undergradstudent/Research/XRB-Analysis/Galaxies/M66/"

# cluster catalogs from the PHANGS catalogs
cluster_cataglog_wfc3 = pd.read_csv(chandra_hst_dir+"M66_cluster_catalog_uvis_fk5.frame")
compact_assoc_16pc = pd.read_csv(chandra_hst_dir+"assoc_catalog_ws16pc.frame")           # uses 16 pc watershed algorithm 
compact_assoc_acs = pd.read_csv(chandra_hst_dir+"M66_assoc_catalog_acs-uvis.frame")

daoclean = remove_unnamed(pd.read_csv(chandra_hst_dir+"M66_daoclean_matches.frame"))
M66_notes = remove_unnamed(pd.read_csv(chandra_hst_dir+"M66_XRB_notes.txt"))

# hdu for euclidean distance
f555w = hstdir+"M66_mosaic_uvis_f555w_drc_sci.fits"

In [104]:
# Only crossref sources that correspond to an XRB with an optical counterpart
M66_hmxbs = M66_notes.query('Class == "HMXB"')
best_stars = pd.merge(daoclean, M66_hmxbs, left_on=['CSC ID', 'StarID'],
                      right_on=['CSC ID', 'Best Star'], how='right')

display(best_stars)

,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,F814W ID,RA,Dec,CSC ID,StarID,ID,Class,Best Star,Mass,Notes
0,3743.843919,5833.507868,19247,130956.0,NaN,36055.0,NaN,170.062292,12.992626,2CXO J112014.9+125933,2,CXO003,HMXB,2.0,NaN,NaN
1,3700.317585,5908.701948,19792,NaN,NaN,36916.0,NaN,170.062784,12.993454,2CXO J112015.0+125936,1,CXO006,HMXB,1.0,NaN,NaN
2,3727.624318,5519.646556,16435,NaN,NaN,NaN,NaN,170.062475,12.989172,2CXO J112015.0+125921,1,CXO007,HMXB,1.0,NaN,NaN
3,3652.013236,5540.798551,16667,NaN,NaN,32346.0,NaN,170.063329,12.989405,2CXO J112015.2+125921,2,CXO008,HMXB,2.0,NaN,NaN
4,3834.343555,5918.287693,19852,NaN,NaN,37022.0,NaN,170.061270,12.993559,2CXO J112014.7+125937,1,CXO009,HMXB,1.0,NaN,NaN
5,3694.780748,5952.563551,20110,134991.0,NaN,NaN,NaN,170.062846,12.993936,2CXO J112015.0+125938,1,CXO010,HMXB,1.0,NaN,NaN
6,3595.444164,6102.186159,21187,NaN,NaN,39199.0,20615.0,170.063968,12.995583,2CXO J112015.3+125944,1,CXO014,HMXB,1.0,NaN,Should that be an HMXB?
7,4265.222965,6110.760032,21250,140213.0,105316.0,NaN,20690.0,170.056403,12.995677,2CXO J112013.5+125944,1,CXO019,HMXB,1.0,NaN,Probably ejected from the cluster 7924
8,4518.077121,6197.146311,21971,NaN,NaN,40508.0,21529.0,170.053547,12.996628,2CXO J112012.8+125947,1,CXO022,HMXB,1.0,NaN,NaN
9,3042.724778,6490.637342,24541,NaN,116238.0,44454.0,24097.0,170.070211,12.999858,2CXO J112016.8+125959,3,CXO024,HMXB,3.0,NaN,A cluster is very nearby


In [78]:
def get_coords(df, regions, catalogs):
    '''A helper function to extract the coordinates and IDs of clusters
    and merge it with the `Crossref`'d dataframe.'''
    for region, catalog in zip(regions, catalogs):
        temp = pd.DataFrame()
        ids = GetIDs(region, verbose=False)
        ra, dec = GetCoords(region, verbose=False)
        temp[f'{catalog} RA'] = ra
        temp[f'{catalog} Dec'] = dec
        temp[f'{catalog} ID'] = ids
        temp[f'{catalog} ID'] = temp[f'{catalog} ID'].astype(float)

        df = pd.merge(
            df,
            temp,
            how='left',
            on=f'{catalog} ID'
        )

    return df

def calculate_distances(
        df,
        regions,
        catalogs,
        sourceid='CSC ID',
        search_radius=0.005,
        coordsys='fk5',
        coordheads=['RA', 'Dec'],
        outfile='XRB_to_cluster.txt',
        calculate_velocity=True,
):
    '''Calculate the distance between XRBs and clusters.'''
    crossref_df = Crossref(
        df=df,
        regions=regions,
        catalogs=catalogs,
        sourceid=sourceid,
        search_radius=search_radius,
        coordsys=coordsys,
        coordheads=coordheads,
        outfile=outfile,
    )

    crossref_df = get_coords(crossref_df, regions, catalogs)


    if calculate_velocity:
        pass

    return crossref_df

In [4]:
# regions to crossreference
regions = [
    chandra_hst_dir+"M66_cluster_catalog_fk5.reg",
    # chandra_hst_dir+"M66_assoc1_catalog_ws16pc_fk5.reg",
    # chandra_hst_dir+"M66_assoc_catalog_acs-uvis_fk5.reg"
]

catalogs = ['cluster wfc3', 
            # 'CA wfc3', 'CA acs'
            ]

search_radius = 0.00005

crossref = calculate_distances(
    df=best_stars,
    catalogs=catalogs,
    regions=regions,
    search_radius=search_radius,
)
print("This is the crossref'd df")
display(crossref)

Finding cross-references between sources. This will take a few minutes. Please wait.. 
DONE WITH CLEANING. CREATING DATAFRAME...
This is the crossref'd df


,RA,Dec,CSC ID,cluster wfc3 ID,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,F814W ID,StarID,ID,Class,Best Star,Mass,Notes,cluster wfc3 RA,cluster wfc3 Dec
0,170.062466,12.991505,2CXO J112014.9+125929,NaN,3728.392237,5731.691614,18393,NaN,95500.0,34799.0,17413.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,170.062444,12.991539,2CXO J112014.9+125929,NaN,3730.382900,5734.693669,18431,NaN,NaN,NaN,17413.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,170.062890,12.991310,2CXO J112015.0+125928,NaN,3690.922608,5713.903685,18254,NaN,95118.0,34583.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,170.062862,12.991389,2CXO J112015.0+125928,NaN,3693.339708,5721.089782,18303,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,170.062260,12.992573,2CXO J112014.9+125933,NaN,3746.714500,5828.653600,19206,NaN,NaN,35989.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,170.062011,13.020465,2CXO J112014.9+130113,NaN,3768.771693,8363.063045,34886,203812.0,157738.0,59483.0,NaN,1,CXO057,HMXB,1.0,NaN,NaN,NaN,NaN
258,170.054162,12.960218,2CXO J112013.0+125736,NaN,4463.710840,2888.783967,1804,48203.0,NaN,NaN,1718.0,1,CXO060,HMXB,1.0,NaN,NaN,NaN,NaN
259,170.066447,12.957722,2CXO J112015.9+125727,NaN,3375.849092,2661.999469,1476,NaN,25603.0,5443.0,1413.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,170.066447,12.957722,2CXO J112015.9+125727,NaN,3375.849092,2661.999469,1476,NaN,25753.0,NaN,NaN,2,CXO061,HMXB,2.0,NaN,NaN,NaN,NaN


In [5]:
matches = crossref.query('`cluster wfc3 RA`.notnull()').reset_index(drop=True)
print("Printing matches")
display(matches.query('`cluster wfc3 ID`.notnull()'))
WriteReg(
    sources=matches,
    outfile='/Users/undergradstudent/Downloads/test_calc_distance.reg',
    coordsys='fk5',
    coordheads=['RA', 'Dec'],
    reg_type='ruler',
    additional_coords=['cluster wfc3 RA', 'cluster wfc3 Dec'],
    idheader='CSC ID',
    radunit='arcsec',
)

Printing matches


,RA,Dec,CSC ID,cluster wfc3 ID,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,F814W ID,StarID,ID,Class,Best Star,Mass,Notes,cluster wfc3 RA,cluster wfc3 Dec
0,170.073396,12.982320,2CXO J112017.6+125856,5530,2760.635455,4897.088811,11581,NaN,77504.0,24692.0,10815.0,6,NaN,NaN,NaN,NaN,NaN,170.073410,12.982327
1,170.076278,12.983566,2CXO J112018.3+125900,5876,2505.543551,5010.300555,12590,NaN,79727.0,26196.0,11697.0,1,NaN,NaN,NaN,NaN,NaN,170.076285,12.983565
2,170.055871,13.007273,2CXO J112013.4+130026,9731,4312.345427,7164.339340,30975,NaN,134497.0,53419.0,29773.0,4,NaN,NaN,NaN,NaN,NaN,170.055875,13.007270


Saving /Users/undergradstudent/Downloads/test_calc_distance.reg
/Users/undergradstudent/Downloads/test_calc_distance.reg saved!


In [6]:
matches['RA'][0], matches['Dec'][0], matches['cluster wfc3 RA'][0], matches['cluster wfc3 Dec'][0]

(np.float64(170.07339648511245),
 np.float64(12.982319892660833),
 np.float64(170.07340952643375),
 np.float64(12.982326508066677))

In [7]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

hdu = fits.open(f555w)
wcs = WCS(hdu[0].header)
ra, dec = matches['RA'].values, matches['Dec'].values

# Convert distances to pixel coords
x, y = SkyCoord(ra, dec, frame='fk5', unit='deg').to_pixel(wcs)
arr = np.array([x, y]).T

# Extracting cluster coordinates
cluster_ra, cluster_dec = matches['cluster wfc3 RA'], matches['cluster wfc3 Dec']
cluster_x, cluster_y = SkyCoord(cluster_ra, cluster_dec,
                                frame='fk5', unit='deg').to_pixel(wcs)
cluster_arr = np.array([cluster_x, cluster_y]).T
# calculate euclidean norm
dist = [np.linalg.norm(cluster_arr[i] - arr[i]) for i in range(len(cluster_arr))]
dist

[np.float64(1.301776054518057),
 np.float64(0.6989995279027877),
 np.float64(0.44334101436207224)]

In [8]:
dist[0]

pixtoarcs = 0.03962 # arcsec/pix
dist1 = dist[0] * pixtoarcs
dist1

np.float64(0.05157636728000542)

## Calculate the euclidean distance

The distances are not as exact as found on ds9. I am not exactly sure why that is happening. I think it is likely because of how these values are stored in pandas dataframes and numpy arrays. I will need to look more into them to see what is happening under the hood.

In [9]:
def euclidean_distance(hdu, df, catalogs, frame='fk5', unit='deg',):
    try: wcs = WCS(hdu['SCI'].header)
    except: wcs = WCS(hdu['PRIMARY'].header)

    # If the dataframe contains the x and y coordinates
    if 'X' and 'Y' in df.columns:
        x, y = df['X'].values, df['Y'].values
    else: 
        ra, dec = df['RA'].values, df['Dec'].values
        x, y = SkyCoord(ra, dec, frame=frame, unit=unit).to_pixel(wcs)

    arr = np.array([x, y]).T
    # Extract the coordinates that are going to be used to calculate 
    # the distance to the first object
    for catalog in catalogs:
        if (f'{catalog} X' and f'{catalog} Y') in df.columns:
            x1, y1 = df[f'{catalog} X'].values, df[f'{catalog} Y'].values
        else:
            ra1, dec1 = df[f'{catalog} RA'].values, df[f'{catalog} Dec'].values
            x1, y1 = SkyCoord(ra1, dec1, frame=frame, unit=unit).to_pixel(wcs)

        # the comparison array contains the coordinates which will calculate
        # the distance of the first object to these objects.
        cmpr_arr = np.array([x1, y1]).T

        dist = [np.linalg.norm(arr[i] - cmpr_arr[i]) for i in range(len(df))]

        print(dist)

In [10]:
euclidean_distance(hdu, matches, catalogs)

[np.float64(1.6958581199437064), np.float64(1.4605707941932315), np.float64(1.3053588776630054)]


When compared with the distances on ds9, the difference is very big. I am going to comment out using the X and Y coordinates in the dataframe for now.

In [11]:
def euclidean_distance(hdu, df, catalogs, frame='fk5', unit='deg',):
    try: wcs = WCS(hdu['SCI'].header)
    except: wcs = WCS(hdu['PRIMARY'].header)

    # If the dataframe contains the x and y coordinates
    # The code below has been commented because I suspect there is something going
    # wrong with the conversion between the coordinates. This is likely due to how 
    # data is stored within pandas dataframes and numpy arrays. I am still working on
    # how to combat that. Until then, convert the dataframes RA and Dec
    # if 'X' and 'Y' in df.columns:
    #     x, y = df['X'].values, df['Y'].values
    # else: 
    ra, dec = df['RA'].values, df['Dec'].values
    x, y = SkyCoord(ra, dec, frame=frame, unit=unit).to_pixel(wcs)

    arr = np.array([x, y]).T
    # Extract the coordinates that are going to be used to calculate 
    # the distance to the first object
    for catalog in catalogs:
        if (f'{catalog} X' and f'{catalog} Y') in df.columns:
            x1, y1 = df[f'{catalog} X'].values, df[f'{catalog} Y'].values
        else:
            ra1, dec1 = df[f'{catalog} RA'].values, df[f'{catalog} Dec'].values
            x1, y1 = SkyCoord(ra1, dec1, frame=frame, unit=unit).to_pixel(wcs)

        # the comparison array contains the coordinates which will calculate
        # the distance of the first object to these objects.
        cmpr_arr = np.array([x1, y1]).T

        dist = [np.linalg.norm(arr[i] - cmpr_arr[i]) for i in range(len(df))]

        print(dist)

In [11]:
euclidean_distance(hdu, matches, catalogs)

[np.float64(1.301776054518057), np.float64(0.6989995279027877), np.float64(0.44334101436207224)]


These look more accurate when compared with ds9.

In [13]:
def euclidean_distance(hdu,
                       df,
                       catalogs, 
                       frame='fk5', 
                       unit_of_coords='deg', 
                       unit_of_dist='pix',
                       instrument=None,
                       pixtoarcs=None,
                       shorten_df=False,
                       additional_cols=[]
):
    df = df.copy()
    hdu = fits.open(hdu)
    try: wcs = WCS(hdu['SCI'].header)
    except: wcs = WCS(hdu['PRIMARY'].header)

    # If the dataframe contains the x and y coordinates
    # The code below has been commented because I suspect there is something going
    # wrong with the conversion between the coordinates. This is likely due to how 
    # data is stored within pandas dataframes and numpy arrays. I am still working on
    # how to combat that. Until then, convert the dataframes RA and Dec
    # if 'X' and 'Y' in df.columns:
    #     x, y = df['X'].values, df['Y'].values
    # else: 
    ra, dec = df['RA'].values, df['Dec'].values
    x, y = SkyCoord(ra, dec, frame=frame, unit=unit_of_coords).to_pixel(wcs)

    arr = np.array([x, y]).T

    object_cols = []

    # Extract the coordinates that are going to be used to calculate 
    # the distance to the first object
    for catalog in catalogs:
        if (f'{catalog} X' and f'{catalog} Y') in df.columns:
            x1, y1 = df[f'{catalog} X'].values, df[f'{catalog} Y'].values
        else:
            ra1, dec1 = df[f'{catalog} RA'].values, df[f'{catalog} Dec'].values
            x1, y1 = SkyCoord(ra1, dec1, frame=frame, unit=unit_of_coords).to_pixel(wcs)

        # the comparison array contains the coordinates which will calculate
        # the distance of the first object to these objects.
        object_arr = np.array([x1, y1]).T

        dist = np.array([np.linalg.norm(arr[i] - object_arr[i]) for i in range(len(df))])
        # Incorporate unit conversion to also include arcsecs
        if unit_of_dist == 'arcsec' or :
            if instrument:
                if instrument.lower() == 'acs':
                    pixtoarcs = 0.05
                elif instrument.lower() == 'wfc3':
                    pixtoarcs = 0.03962
                    dist = dist * pixtoarcs
                elif instrument.lower() == 'nircaml': # if Nircam long wavelength
                    pixtoarcs = 0.063
                    dist = dist * pixtoarcs
                elif instrument.lower() == 'nircams': # if Nircam short wavelength
                    pixtoarcs = 0.031
                    dist = dist * pixtoarcs
            elif pixtoarcs:
                    if not pixtoarcs: input("Please input pixtoarcs") 
                    dist = dist * pixtoarcs

        df[f'Distance to {catalog} ({unit_of_dist})'] = dist
        object_id = f'{catalog} ID'
        object_ra = f'{catalog} RA'
        object_dec = f'{catalog} Dec'
        object_unit = f'Distance to {catalog} ({unit_of_dist})'
        object_cols.extend([object_id, object_ra, object_dec, object_unit ])

    if shorten_df: 
        cols = ['CSC ID', 'X', 'Y', 'RA', 'Dec'] + object_cols + additional_cols
        df = df[cols].reset_index(drop=True)

    return df

SyntaxError: invalid syntax (1347742851.py, line 47)

In [14]:
euclidean_distance(hdu, matches, catalogs, shorten_df=True, unit_of_dist='arcsec', instrument='nircamS')

TypeError: euclidean_distance() got an unexpected keyword argument 'shorten_df'

The code above still needs a lot of updates. I plan on creating a class and then I can perform all these operations on those classes. Additionally, it will likely not behave well if more than 1 dataframe is passed to calculate distances

## Calculate the Velocity of ejection

In [15]:
compact_assoc_16pc

,Unnamed: 0,reg_id,reg_x,reg_y,RA,Dec,reg_ra,reg_dec,reg_area,reg_rad,...,V_dolmag_vega_err,I_dolmag_vega,I_dolmag_vega_err,reg_dolflux_Age_MinChiSq,reg_dolflux_Age_MinChiSq_err,reg_dolflux_Mass_MinChiSq,reg_dolflux_Mass_MinChiSq_err,reg_dolflux_Ebv_MinChiSq,reg_dolflux_Ebv_MinChiSq_err,reg_dolflux_ChiSq_Reduced
0,0,1.0,7386.0,2975.0,170.057439,12.936646,170.057434,12.936649,523.132513,12.904195,...,0.031663,22.971668,0.049561,621.0,96.0,10247.243218,2201.758,0.00,0.03,0.811458
1,1,2.0,7229.0,3023.0,170.059212,12.937174,170.059206,12.937177,327.428264,10.208999,...,0.031586,23.495558,0.046549,537.0,213.0,5990.247664,55.648,0.01,0.13,1.597546
2,2,3.0,7312.0,3201.0,170.058274,12.939133,170.058269,12.939136,323.664720,10.150157,...,0.072645,24.033986,0.131829,717.0,180.0,4182.178484,1338.572,0.00,0.18,1.789404
3,3,4.0,7434.0,3212.0,170.056897,12.939254,170.056892,12.939257,331.191807,10.267503,...,0.086280,23.604647,0.072242,37.0,58.0,3063.127299,4181.037,0.56,0.12,0.319823
4,4,5.0,7646.0,3233.0,170.054503,12.939485,170.054498,12.939488,455.388734,12.039715,...,0.053946,23.555929,0.070665,118.0,114.0,4061.832217,2807.382,0.25,0.44,0.045098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4239,4239,4240.0,5720.0,10892.0,170.076257,13.023776,170.076251,13.023780,413.989759,11.479418,...,0.034623,23.510354,0.069254,5.0,1.0,561.174598,124.246,0.17,0.08,1.047496
4240,4240,4241.0,6509.0,10905.0,170.067344,13.023920,170.067339,13.023923,323.664720,10.150157,...,0.084728,23.697308,0.090891,956.0,628.0,6210.304618,3321.105,0.00,0.00,0.165169
4241,4241,4242.0,5967.0,10958.0,170.073467,13.024503,170.073461,13.024506,350.009523,10.555164,...,0.117432,24.233814,0.144884,1.0,4.0,1919.652048,1477.328,0.68,0.18,0.817480
4242,4242,4243.0,5445.0,11002.0,170.079363,13.024987,170.079358,13.024990,331.191807,10.267503,...,0.052922,22.677239,0.041704,147.0,1682.0,14089.468617,9475.585,0.47,0.47,0.248801


This cluster catalog does not have ages. Will need to use a different catalog for now.

In [17]:
compact_assoc = compact_assoc_16pc[['reg_id', 'reg_dolflux_Age_MinChiSq',
       'reg_dolflux_Age_MinChiSq_err']].rename(
           columns={'reg_dolflux_Age_MinChiSq' : 'Age', 
           'reg_dolflux_Age_MinChiSq_err' : 'Age Err'}
       )
compact_assoc.head()

,reg_id,Age,Age Err
0,1.0,621.0,96.0
1,2.0,537.0,213.0
2,3.0,717.0,180.0
3,4.0,37.0,58.0
4,5.0,118.0,114.0


In [ ]:
def euclidean_distance(filename,
                       df,
                       catalogs, 
                       instrument,
                       frame='fk5', 
                       unit_of_coords='deg', 
                       unit_of_dist='pc',
                       arcsectopc=45.4,
                       shorten_df=False,
                       additional_cols=[]
):
    '''Calculate euclidean distance between two sets of objects
    
    Parameters
    ----------
    filename : str
        The path of the base image to be used for distance calculation.
    df : pd.DataFrame
        Dataframe containing the coordinates of the objects to compare
    catalogs : list
        list containing the names of the objects being compared to.
        Default is 'fk5'
    instrument : str
    The instrument of the base image. Required to convert coordinates to coordinates
    other than pixels. Default is `None`. Other options include 'wfc3', 'acs', 
    'nircamL' for long wavelength with NIRCam and 'nircamS' for short wavelength
    with NIRCam.
    frame : str
        The reference coordinate frame of the object. Will be used to 
        convert coordinates to pixels. Default is 'fk5.
    unit_of_coords : str
        The units of the coordinates that are being extracted from the dataframe.
        Default is 'deg'
    unit_of_dist : str
        The units to use in the distances between the objects. Default is 'pix'.
    arcsectopc : float
        The arcsec to parsec conversion to use for changing coordinates to coordinates. Default is 45.4''
        for the NGC 3627 galaxy.
        This parameter is not required to pass usually as the
        `instrument` parameter uses the `pixtoarcs` conversion based upon the instrument being used.
    shorten_df : bool
        If `True`, provides a smaller dataframe containing only the CSC ID, coordinates (image and others)
        as well as the distances. Default is False
    additional_cols : list of strings
        Additional columns to include in the shortened dataframe

    Returns
    -------
    df : pd.Dataframe
        Dataframe containing the distances between the objects
    '''
    df = df.copy()
    hdu = fits.open(filename)
    try: wcs = WCS(hdu['SCI'].header)
    except: wcs = WCS(hdu['PRIMARY'].header)

    # If the dataframe contains the x and y coordinates
    # The code below has been commented because I suspect there is something going
    # wrong with the conversion between the coordinates. This is likely due to how 
    # data is stored within pandas dataframes and numpy arrays. I am still working on
    # how to combat that. Until then, convert the dataframes RA and Dec
    # if 'X' and 'Y' in df.columns:
    #     x, y = df['X'].values, df['Y'].values
    # else: 
    ra, dec = df['RA'].values, df['Dec'].values
    x, y = SkyCoord(ra, dec, frame=frame, unit=unit_of_coords).to_pixel(wcs)

    arr = np.array([x, y]).T

    object_cols = []

    # Extract the coordinates that are going to be used to calculate 
    # the distance to the first object
    object_cols = []
    for catalog in catalogs:
        if (f'{catalog} X' and f'{catalog} Y') in df.columns:
            x1, y1 = df[f'{catalog} X'].values, df[f'{catalog} Y'].values
        else:
            ra1, dec1 = df[f'{catalog} RA'].values, df[f'{catalog} Dec'].values
            x1, y1 = SkyCoord(ra1, dec1, frame=frame, unit=unit_of_coords).to_pixel(wcs)

        # the comparison array contains the coordinates which will calculate
        # the distance of the first object to these objects.
        object_arr = np.array([x1, y1]).T

        dist = np.array([np.linalg.norm(arr[i] - object_arr[i]) for i in range(len(df))])
        # Incorporate unit conversion to also include arcsecs
        if unit_of_dist == 'arcsec':
            dist = dist * instrument_pixtoarcs[instrument]
        if unit_of_dist == 'pc' or unit_of_dist == 'parsec':
            dist = dist * instrument_pixtoarcs[instrument] * arcsectopc

        df[f'Distance ({unit_of_dist})'] = dist
        object_id = f'{catalog} ID'
        object_ra = f'{catalog} RA'
        object_dec = f'{catalog} Dec'
        object_dist = f'Distance ({unit_of_dist})'
        object_cols.extend([object_id, object_ra, object_dec, object_dist])
        print(object_cols)
    if shorten_df: 
        cols = ['CSC ID', 'X', 'Y', 'RA', 'Dec'] + object_cols + additional_cols
        df = df[cols].reset_index(drop=True)

    return df

In [22]:
matches

,RA,Dec,CSC ID,CA wfc3 ID,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,F814W ID,StarID,ID,Class,Best Star,Mass,Notes,CA wfc3 RA,CA wfc3 Dec
0,170.062890,12.991310,2CXO J112015.0+125928,2733.0,3690.922608,5713.903685,18254,NaN,95118.0,34583.0,NaN,1,NaN,NaN,NaN,NaN,NaN,170.062899,12.991303
1,170.062862,12.991389,2CXO J112015.0+125928,2739.0,3693.339708,5721.089782,18303,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,170.062877,12.991435
2,170.064034,12.991722,2CXO J112015.2+125932X,2752.0,3589.611349,5751.360661,18558,NaN,NaN,NaN,17527.0,45,NaN,NaN,NaN,NaN,NaN,170.064040,12.991699
3,170.064034,12.991722,2CXO J112015.2+125932X,2752.0,3589.611349,5751.360661,18558,NaN,NaN,NaN,17574.0,46,NaN,NaN,NaN,NaN,NaN,170.064040,12.991699
4,170.062932,12.992449,2CXO J112015.2+125932X,2791.0,3687.207643,5817.461704,19074,NaN,97677.0,NaN,NaN,120,NaN,NaN,NaN,NaN,NaN,170.062922,12.992458
5,170.062991,12.992617,2CXO J112015.2+125932X,2801.0,3681.976976,5832.677619,19246,NaN,98149.0,36041.0,NaN,138,NaN,NaN,NaN,NaN,NaN,170.062990,12.992634
6,170.065377,12.979318,2CXO J112015.7+125845,1698.0,3470.638926,4624.306397,9260,NaN,73613.0,21521.0,8708.0,1,NaN,NaN,NaN,NaN,NaN,170.065373,12.979329
7,170.073316,12.982390,2CXO J112017.6+125856,2071.0,2767.745789,4903.458676,11642,NaN,77605.0,24779.0,10859.0,9,NaN,NaN,NaN,NaN,NaN,170.073290,12.982366
8,170.073270,12.982398,2CXO J112017.6+125856,2071.0,2771.853078,4904.192542,11650,NaN,77605.0,24780.0,10879.0,10,NaN,NaN,NaN,NaN,NaN,170.073290,12.982366
9,170.073270,12.982398,2CXO J112017.6+125856,2071.0,2771.853078,4904.192542,11650,NaN,77630.0,NaN,NaN,11,NaN,NaN,NaN,NaN,NaN,170.073290,12.982366


In [21]:
# regions to crossreference
regions = [
    # chandra_hst_dir+"M66_cluster_catalog_fk5.reg",
    chandra_hst_dir+"M66_assoc1_catalog_ws16pc_fk5.reg",
    # chandra_hst_dir+"M66_assoc_catalog_acs-uvis_fk5.reg"
]

catalogs = [
    # 'cluster wfc3', 
    'CA wfc3', 
    # 'CA acs'
            ]

search_radius = 0.00005

crossref = calculate_distances(
    df=best_stars,
    catalogs=catalogs,
    regions=regions,
    search_radius=search_radius,
)
print("This is the crossref'd df")
# display(crossref)

matches = crossref.query('`CA wfc3 RA`.notnull()').reset_index(drop=True)
print("Printing matches")
matches

Finding cross-references between sources. This will take a few minutes. Please wait.. 
DONE WITH CLEANING. CREATING DATAFRAME...
This is the crossref'd df
Printing matches


,RA,Dec,CSC ID,CA wfc3 ID,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,F814W ID,StarID,ID,Class,Best Star,Mass,Notes,CA wfc3 RA,CA wfc3 Dec
0,170.062890,12.991310,2CXO J112015.0+125928,2733.0,3690.922608,5713.903685,18254,NaN,95118.0,34583.0,NaN,1,NaN,NaN,NaN,NaN,NaN,170.062899,12.991303
1,170.062862,12.991389,2CXO J112015.0+125928,2739.0,3693.339708,5721.089782,18303,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,170.062877,12.991435
2,170.064034,12.991722,2CXO J112015.2+125932X,2752.0,3589.611349,5751.360661,18558,NaN,NaN,NaN,17527.0,45,NaN,NaN,NaN,NaN,NaN,170.064040,12.991699
3,170.064034,12.991722,2CXO J112015.2+125932X,2752.0,3589.611349,5751.360661,18558,NaN,NaN,NaN,17574.0,46,NaN,NaN,NaN,NaN,NaN,170.064040,12.991699
4,170.062932,12.992449,2CXO J112015.2+125932X,2791.0,3687.207643,5817.461704,19074,NaN,97677.0,NaN,NaN,120,NaN,NaN,NaN,NaN,NaN,170.062922,12.992458
5,170.062991,12.992617,2CXO J112015.2+125932X,2801.0,3681.976976,5832.677619,19246,NaN,98149.0,36041.0,NaN,138,NaN,NaN,NaN,NaN,NaN,170.062990,12.992634
6,170.065377,12.979318,2CXO J112015.7+125845,1698.0,3470.638926,4624.306397,9260,NaN,73613.0,21521.0,8708.0,1,NaN,NaN,NaN,NaN,NaN,170.065373,12.979329
7,170.073316,12.982390,2CXO J112017.6+125856,2071.0,2767.745789,4903.458676,11642,NaN,77605.0,24779.0,10859.0,9,NaN,NaN,NaN,NaN,NaN,170.073290,12.982366
8,170.073270,12.982398,2CXO J112017.6+125856,2071.0,2771.853078,4904.192542,11650,NaN,77605.0,24780.0,10879.0,10,NaN,NaN,NaN,NaN,NaN,170.073290,12.982366
9,170.073270,12.982398,2CXO J112017.6+125856,2071.0,2771.853078,4904.192542,11650,NaN,77630.0,NaN,NaN,11,NaN,NaN,NaN,NaN,NaN,170.073290,12.982366


In [23]:
instrument_pixtoarcs = {
    'acs' : 0.05,
    'wfc3' : 0.03962,
    'nircaml' : 0.063,
    'nircams' : 0.031,
}

arcsectopc = 45.4 # https://iopscience.iop.org/article/10.3847/1538-4357/ace162



In [24]:
dist1 = euclidean_distance(
    f555w,
    df=matches,
    instrument='wfc3',
    catalogs=catalogs,
    shorten_df=True
)
dist1

['CA wfc3 ID', 'CA wfc3 RA', 'CA wfc3 Dec', 'Distance (pc)']


,CSC ID,X,Y,RA,Dec,CA wfc3 ID,CA wfc3 RA,CA wfc3 Dec,Distance (pc)
0,2CXO J112015.0+125928,3690.922608,5713.903685,170.062890,12.991310,2733.0,170.062899,12.991303,1.904140
1,2CXO J112015.0+125928,3693.339708,5721.089782,170.062862,12.991389,2739.0,170.062877,12.991435,7.848400
2,2CXO J112015.2+125932X,3589.611349,5751.360661,170.064034,12.991722,2752.0,170.064040,12.991699,3.886200
3,2CXO J112015.2+125932X,3589.611349,5751.360661,170.064034,12.991722,2752.0,170.064040,12.991699,3.886200
4,2CXO J112015.2+125932X,3687.207643,5817.461704,170.062932,12.992449,2791.0,170.062922,12.992458,2.132238
5,2CXO J112015.2+125932X,3681.976976,5832.677619,170.062991,12.992617,2801.0,170.062990,12.992634,2.863804
6,2CXO J112015.7+125845,3470.638926,4624.306397,170.065377,12.979318,1698.0,170.065373,12.979329,1.895406
7,2CXO J112017.6+125856,2767.745789,4903.458676,170.073316,12.982390,2071.0,170.073290,12.982366,5.741590
8,2CXO J112017.6+125856,2771.853078,4904.192542,170.073270,12.982398,2071.0,170.073290,12.982366,6.152244
9,2CXO J112017.6+125856,2771.853078,4904.192542,170.073270,12.982398,2071.0,170.073290,12.982366,6.152244


In [25]:
matches = matches[-5:]
distances = euclidean_distance(f555w,
                               df=matches,
                               instrument='wfc3',
                               catalogs=catalogs,
                               shorten_df=True,)

distances

['CA wfc3 ID', 'CA wfc3 RA', 'CA wfc3 Dec', 'Distance (pc)']


,CSC ID,X,Y,RA,Dec,CA wfc3 ID,CA wfc3 RA,CA wfc3 Dec,Distance (pc)
0,2CXO J112019.1+125947,2190.213719,6174.760583,170.079840,12.996381,2954.0,170.079842,12.996398,2.737512
1,2CXO J112019.1+125947,2188.079548,6178.485934,170.079864,12.996422,2954.0,170.079842,12.996398,5.361518
2,2CXO J112019.1+125947,2191.607801,6178.359825,170.079824,12.996421,2954.0,170.079842,12.996398,4.646398
3,2CXO J112019.1+125947,2191.607801,6178.359825,170.079824,12.996421,2954.0,170.079842,12.996398,4.646398
4,2CXO J112013.2+130035,4363.615697,7398.711500,170.055292,13.009852,3999.0,170.055297,13.009847,1.246037


For the calculation of the velocity of ejection, we will be using the formula,
$$\text{(min) Velocity of ejection} = \frac{\text{distance between XRBs and clusters}}{\text{(max) Age of the cluster}}$$

In [106]:
M66_best = pd.read_csv(chandra_hst_dir + "M66_csc_bestrads.frame")

In [107]:
M66_best = M66_best[['CSC ID', '2Sig']]


In [108]:
distances.columns

Index(['CSC ID', 'X', 'Y', 'RA', 'Dec', 'CA wfc3 ID', 'CA wfc3 RA',
       'CA wfc3 Dec', 'Distance to CA wfc3 (pc)'],
      dtype='object')

In [123]:
confirmed = pd.merge(distances, compact_assoc, left_on='CA wfc3 ID', right_on='reg_id', how='left')
confirmed = pd.merge(confirmed, M66_best, on='CSC ID', how='left')
confirmed

,CSC ID,X,Y,RA,Dec,CA wfc3 ID,CA wfc3 RA,CA wfc3 Dec,Distance to CA wfc3 (pc),reg_id,Age,Age Err,2Sig
0,2CXO J112019.1+125947,2190.213719,6174.760583,170.079840,12.996381,2954.0,170.079842,12.996398,2.737512,2954.0,2.0,1.0,0.814304
1,2CXO J112019.1+125947,2188.079548,6178.485934,170.079864,12.996422,2954.0,170.079842,12.996398,5.361518,2954.0,2.0,1.0,0.814304
2,2CXO J112019.1+125947,2191.607801,6178.359825,170.079824,12.996421,2954.0,170.079842,12.996398,4.646398,2954.0,2.0,1.0,0.814304
3,2CXO J112019.1+125947,2191.607801,6178.359825,170.079824,12.996421,2954.0,170.079842,12.996398,4.646398,2954.0,2.0,1.0,0.814304
4,2CXO J112013.2+130035,4363.615697,7398.711500,170.055292,13.009852,3999.0,170.055297,13.009847,1.246037,3999.0,6.0,4.0,0.560363


In [127]:
df = confirmed
df['Velocity'] = df['Distance to CA wfc3 (pc)'] / df['Age']
v = df['Velocity']
t, t_err = df['Age'].values, df['Age Err'].values
d, d_err = df['Distance to CA wfc3 (pc)'].values, df['2Sig'].values
df['Velocity Err'] =  v * np.sqrt((t_err / t) ** 2 + (d_err / d) ** 2)
df

,CSC ID,X,Y,RA,Dec,CA wfc3 ID,CA wfc3 RA,CA wfc3 Dec,Distance to CA wfc3 (pc),reg_id,Age,Age Err,2Sig,Velocity,Velocity Err
0,2CXO J112019.1+125947,2190.213719,6174.760583,170.079840,12.996381,2954.0,170.079842,12.996398,2.737512,2954.0,2.0,1.0,0.814304,1.368756,0.796333
1,2CXO J112019.1+125947,2188.079548,6178.485934,170.079864,12.996422,2954.0,170.079842,12.996398,5.361518,2954.0,2.0,1.0,0.814304,2.680759,1.400853
2,2CXO J112019.1+125947,2191.607801,6178.359825,170.079824,12.996421,2954.0,170.079842,12.996398,4.646398,2954.0,2.0,1.0,0.814304,2.323199,1.230888
3,2CXO J112019.1+125947,2191.607801,6178.359825,170.079824,12.996421,2954.0,170.079842,12.996398,4.646398,2954.0,2.0,1.0,0.814304,2.323199,1.230888
4,2CXO J112013.2+130035,4363.615697,7398.711500,170.055292,13.009852,3999.0,170.055297,13.009847,1.246037,3999.0,6.0,4.0,0.560363,0.207673,0.167004


In [91]:
from XRBID.Sources import GetIDs, GetCoords

def get_coords(df, regions, catalogs):
    '''A helper function to extract the coordinates and IDs of clusters
    and merge it with the `Crossref`'d dataframe.'''
    for region, catalog in zip(regions, catalogs):
        temp = pd.DataFrame()
        ids = GetIDs(region, verbose=False)
        ra, dec = GetCoords(region, verbose=False)
        temp[f'{catalog} RA'] = ra
        temp[f'{catalog} Dec'] = dec
        temp[f'{catalog} ID'] = ids
        temp[f'{catalog} ID'] = temp[f'{catalog} ID'].astype(float)

        df = pd.merge(
            df,
            temp,
            how='left',
            on=f'{catalog} ID'
        )

    return df

class XrayBinary:
    def __init__(self, df):
        self.df = df.copy()
        if 'X' in self.df.columns:
            self.x = self.df['X']
        if 'Y' in self.df.columns:
            self.y = self.df['Y']
        if 'RA' in self.df.columns:
            self.ra = self.df['RA']
        if 'Dec' in self.df.columns:
            self.dec = self.df['Dec']

    def _repr_html_(self):
        return self.df._repr_html_()

    def __repr__(self):
        return self.df.__repr__() # Or self.df.to_string() for full display

    def __str__(self):
        return self.df.__str__() # Or self.df.to_string() for full display
    
    def crossref(
            self,
            cluster_region,
            cluster_name,
            sourceid='CSC ID',
            search_radius=0.005,
            coordsys='fk5',
            coordheads=['RA', 'Dec'],
            outfile='XRB_to_cluster.txt',
        ):
        self.df = Crossref(
            df=self.df,
            regions=cluster_region,
            catalogs=cluster_name,
            sourceid=sourceid,
            search_radius=search_radius,
            coordsys=coordsys,
            coordheads=coordheads,
            outfile=outfile,
        )

        return self.df

    def calculate_distance(
            self,
            filename,
            instrument,
            cluster_region,
            cluster_name,
            search_radius=0.0005,
            sourceid='CSC ID',
            coordsys='fk5',
            coordheads=['RA', 'Dec'],
            outfile='XRB_to_cluster.txt',
            frame='fk5',
            unit_of_coords='deg',
            unit_of_dist='pc',
            arcsectopc=45.4,
            shorten_df=False,
            additional_cols=[],
            calculate_velocity=False,
            velocity_coords=['Distance', 'Age'],
            calc_velocity_err=False,
            velocity_err_headers=['Distance Err', 'Age Err']
    ):
        self.crossref(
            cluster_region=cluster_region,
            cluster_name=cluster_name,
            sourceid=sourceid,
            search_radius=search_radius,
            coordsys=coordsys,
            coordheads=coordheads,
            outfile=outfile,

        )

        self._get_coords(cluster_region, cluster_name)

        self._euclidean_distance(
            filename,
            cluster_name,
            instrument,
            frame,
            unit_of_coords,
            unit_of_dist,
            arcsectopc,
            shorten_df,
            additional_cols
        )
        if calculate_velocity:
            self.calculate_velocity(
            headers=velocity_coords,
            calc_err=calc_velocity_err,
            err_headers=velocity_err_headers
            )

        self.df = self.df.query(f'`{cluster_name[0]} ID`.notnull()').reset_index(drop=True)
        return self.df
    
    def _get_coords(self, cluster_region, cluster_name):
        self.df = get_coords(self.df, cluster_region, cluster_name)
        return self.df
            

    def _euclidean_distance(
            self,
            filename,
            cluster_name,
            instrument,
            frame='fk5',
            unit_of_coords='deg',
            unit_of_dist='pc',
            arcsectopc=45.4,
            shorten_df=False,
            additional_cols=[]
):
        self.df = euclidean_distance(
            filename=filename,
            instrument=instrument,
            df=self.df,
            catalogs=cluster_name,
            frame=frame,
            unit_of_coords=unit_of_coords,
            unit_of_dist=unit_of_dist,
            arcsectopc=arcsectopc,
            shorten_df=shorten_df,
            additional_cols=additional_cols
        )

        return self.df
    
    def calculate_velocity(
            self,
            headers=['Distance', 'Age'],
            calc_err=False,
            err_headers=['Distance Err', 'Age Err']
    ):
        self.df['Velocity'] = self.df[headers[0]] / self.df[headers[1]]
        if calc_err:
            d, d_err = self.df[headers[0]], self.df[err_headers[0]]
            t, t_err = self.df[headers[1]], self.df[err_headers[1]]
            err = np.sqrt((d_err / d) ** 2 + (t_err / t) ** 2)
            self.df['Velocity Err'] = self.df['Velocity'] * err

        return self.df

In [103]:
stars = XrayBinary(best_stars)

stars.calculate_distance(
    f555w,
    instrument='wfc3',
    search_radius=0.00005,
    cluster_region=regions,
    cluster_name=catalogs,
)

# Merge the Age and Age errs into the main df
stars.df = stars.df.merge(compact_assoc, left_on='CA wfc3 ID', right_on='reg_id', how='left')
stars.df = stars.df.drop('reg_id', axis=1)

# Merge the 2 sig into the main df
M66_best = remove_unnamed(pd.read_csv(chandra_hst_dir+"M66_csc_bestrads.frame"))[['CSC ID', '2Sig']]
stars.df = stars.df.merge(M66_best, on='CSC ID', how='left')
stars.df

Finding cross-references between sources. This will take a few minutes. Please wait.. 
DONE WITH CLEANING. CREATING DATAFRAME...
['CA wfc3 ID', 'CA wfc3 RA', 'CA wfc3 Dec', 'Distance (pc)']


,RA,Dec,CSC ID,CA wfc3 ID,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,...,Class,Best Star,Mass,Notes,CA wfc3 RA,CA wfc3 Dec,Distance (pc),Age,Age Err,2Sig
0,170.062890,12.991310,2CXO J112015.0+125928,2733.0,3690.922608,5713.903685,18254,NaN,95118.0,34583.0,...,NaN,NaN,NaN,NaN,170.062899,12.991303,1.904140,770.0,0.0,0.248443
1,170.062862,12.991389,2CXO J112015.0+125928,2739.0,3693.339708,5721.089782,18303,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,170.062877,12.991435,7.848400,717.0,252.0,0.248443
2,170.064034,12.991722,2CXO J112015.2+125932X,2752.0,3589.611349,5751.360661,18558,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,170.064040,12.991699,3.886200,1028.0,200.0,3.451755
3,170.064034,12.991722,2CXO J112015.2+125932X,2752.0,3589.611349,5751.360661,18558,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,170.064040,12.991699,3.886200,1028.0,200.0,3.451755
4,170.062932,12.992449,2CXO J112015.2+125932X,2791.0,3687.207643,5817.461704,19074,NaN,97677.0,NaN,...,NaN,NaN,NaN,NaN,170.062922,12.992458,2.132238,226.0,35.0,3.451755
5,170.062991,12.992617,2CXO J112015.2+125932X,2801.0,3681.976976,5832.677619,19246,NaN,98149.0,36041.0,...,NaN,NaN,NaN,NaN,170.062990,12.992634,2.863804,621.0,156.0,3.451755
6,170.065377,12.979318,2CXO J112015.7+125845,1698.0,3470.638926,4624.306397,9260,NaN,73613.0,21521.0,...,NaN,NaN,NaN,NaN,170.065373,12.979329,1.895406,158.0,11.0,0.557056
7,170.073316,12.982390,2CXO J112017.6+125856,2071.0,2767.745789,4903.458676,11642,NaN,77605.0,24779.0,...,NaN,NaN,NaN,NaN,170.073290,12.982366,5.741590,8.0,1.0,0.553381
8,170.073270,12.982398,2CXO J112017.6+125856,2071.0,2771.853078,4904.192542,11650,NaN,77605.0,24780.0,...,NaN,NaN,NaN,NaN,170.073290,12.982366,6.152244,8.0,1.0,0.553381
9,170.073270,12.982398,2CXO J112017.6+125856,2071.0,2771.853078,4904.192542,11650,NaN,77630.0,NaN,...,NaN,NaN,NaN,NaN,170.073290,12.982366,6.152244,8.0,1.0,0.553381


In [76]:
stars.calculate_velocity(
    headers=['Distance (pc)', 'Age'],
    calc_err=True,
    err_headers=['2Sig', 'Age Err']
)

,RA,Dec,CSC ID,CA wfc3 ID,X,Y,F555W ID,F275W ID,F336W ID,F438W ID,...,Mass,Notes,CA wfc3 RA,CA wfc3 Dec,Distance (pc),Age,Age Err,2Sig,Velocity,Velocity Err
0,170.062890,12.991310,2CXO J112015.0+125928,2733.0,3690.922608,5713.903685,18254,NaN,95118.0,34583.0,...,NaN,NaN,170.062899,12.991303,1.904140,770.0,0.0,0.248443,0.002473,0.000323
1,170.062862,12.991389,2CXO J112015.0+125928,2739.0,3693.339708,5721.089782,18303,NaN,NaN,NaN,...,NaN,NaN,170.062877,12.991435,7.848400,717.0,252.0,0.248443,0.010946,0.003863
2,170.064034,12.991722,2CXO J112015.2+125932X,2752.0,3589.611349,5751.360661,18558,NaN,NaN,NaN,...,NaN,NaN,170.064040,12.991699,3.886200,1028.0,200.0,3.451755,0.003780,0.003437
3,170.064034,12.991722,2CXO J112015.2+125932X,2752.0,3589.611349,5751.360661,18558,NaN,NaN,NaN,...,NaN,NaN,170.064040,12.991699,3.886200,1028.0,200.0,3.451755,0.003780,0.003437
4,170.062932,12.992449,2CXO J112015.2+125932X,2791.0,3687.207643,5817.461704,19074,NaN,97677.0,NaN,...,NaN,NaN,170.062922,12.992458,2.132238,226.0,35.0,3.451755,0.009435,0.015343
5,170.062991,12.992617,2CXO J112015.2+125932X,2801.0,3681.976976,5832.677619,19246,NaN,98149.0,36041.0,...,NaN,NaN,170.062990,12.992634,2.863804,621.0,156.0,3.451755,0.004612,0.005678
6,170.065377,12.979318,2CXO J112015.7+125845,1698.0,3470.638926,4624.306397,9260,NaN,73613.0,21521.0,...,NaN,NaN,170.065373,12.979329,1.895406,158.0,11.0,0.557056,0.011996,0.003623
7,170.073316,12.982390,2CXO J112017.6+125856,2071.0,2767.745789,4903.458676,11642,NaN,77605.0,24779.0,...,NaN,NaN,170.073290,12.982366,5.741590,8.0,1.0,0.553381,0.717699,0.113283
8,170.073270,12.982398,2CXO J112017.6+125856,2071.0,2771.853078,4904.192542,11650,NaN,77605.0,24780.0,...,NaN,NaN,170.073290,12.982366,6.152244,8.0,1.0,0.553381,0.769031,0.118430
9,170.073270,12.982398,2CXO J112017.6+125856,2071.0,2771.853078,4904.192542,11650,NaN,77630.0,NaN,...,NaN,NaN,170.073290,12.982366,6.152244,8.0,1.0,0.553381,0.769031,0.118430
